# Radiate Traces

In [1]:
# install lifelike_gds package if not already installed (e.g. running in Google Colab)
import importlib

if importlib.util.find_spec('lifelike_gds') is None:
  !pip install git+https://github.com/SBRG/GDS-Public

# provide the path to the notebook folder in the github repository in case the notebook is run in Google Colab
github_path = 'SBRG/GDS-Public/main/notebooks/reactome'

In [2]:
import os
import warnings
from pathlib import PurePosixPath

In [3]:
# Import GDS modules
from lifelike_gds.arango_network.radiate_trace import RadiateTrace
from lifelike_gds.arango_network.reactome import *
from lifelike_gds.arango_network.reactome_notebook_functions import *

/Users/dommas/.local/share/virtualenvs/GDS-Public-uKBS7cUg/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# Ignore warnings
warnings.filterwarnings('ignore')

## Settings
Make sure to change to the correct one based on your local settings

In [5]:
# Directory where to look for input data
input_dir = PurePosixPath('input')

# Directory where to output results
output_dir = PurePosixPath('output')
os.makedirs(output_dir, exist_ok=True)

# Reactome arango database connection parameters
# If you are running this notebook from Lifelike's online training BinderHub website,
# then these paremeters are already set for you in the environment.
arango_dbname = os.getenv('REACTOME_ARANGO_DATABASE', 'reactome')

## Parameters
Set parameters
- source_name: name for the source entities. 
- source_file: source file name in input dir


In [6]:
source_name = 'ph-updown-metabolites'

In [7]:
nodes_select_file_path = input_dir / f'Radiate_analysis_for_{source_name}.xlsx'
if os.path.isfile(nodes_select_file_path):
  nodes_select_file_ref = nodes_select_file_path
else:
  # if does not exist localy, pull from github
  nodes_select_file_ref = f'https://raw.githubusercontent.com/{github_path}/{nodes_select_file_path}'

## Connect to arango database

In [8]:
database = ReactomeDB(arango_dbname)

## Load graph from arango graph database to memery

In [9]:
tracegraph = RadiateTrace(Reactome(database))
# set up output directory where the excel and graph files will write to
tracegraph.datadir = output_dir
# initiate tracegraph by loading graph data from arango
# a networkx graph is created here.  
tracegraph.init_default_graph()

INFO: load reactome graph
INFO: MultiDirectedGraph with 71225 nodes and 112575 edges


## Get source nodes, run radiate analysis, and export data to excel

In [10]:
ids_ph = [
    'R-ALL-83957', # Hypoxanthine (extracellular region)
    'R-ALL-3296298', # x12(S)-HETE (extracellular region)
    'R-ALL-210382',	# Glutamic acid (extracellular region)
    'R-ALL-419305',	# Sphingosine 1 phosphate (extracellular region)
    'R-ALL-351977', # Arginine (extracellular region)
    'R-ALL-351970'	# Histidine (extracellular region)
]
source_nodes = database.get_nodes_by_attr(ids_ph, 'stId')
selected_forward_nodes = get_selected_nodes(nodes_select_file_ref, 'pageranks')
selected_reverse_nodes = get_selected_nodes(nodes_select_file_ref, 'reverse pageranks')

HTTPError: HTTP Error 404: Not Found

In [ ]:
export_radiate_traces(database, tracegraph, source_name, source_nodes, selected_forward_nodes, None)
export_radiate_traces(database, tracegraph, source_name, source_nodes, None, selected_reverse_nodes)